In [1]:
import PlottingFunction as pf
import DataManipulation as dm
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
import winsound
from sklearn.mixture import GaussianMixture

%matplotlib qt5

In [3]:
df = pd.read_csv(Path.cwd() / 'data' / 'vast' / "vast_rally_600k.csv")
df = dm.keep_data_in_frame(df, compressed=True, only_half=True)
df = dm.change_vast_error_shots(df,'rally')

# Look at just "winning" shots
vast_min = 0.5
df = df[(df.vast >= vast_min) | (df.vast == 0)]

# Import player regions
receiver_position = 'receiver_start'
receiver_position_title = "Start"
impact_regions, receiver_regions = pf.createPlayerRegions(type = 'rally')

In [4]:
for impact_region in impact_regions:
    for receiver_region in receiver_regions:
               
        df_filter = df
        df_filter = dm.filter_data_by_buckets(df_filter, 'impact', impact_regions[impact_region],compressed=True,only_half=False)
        df_filter = dm.filter_data_by_buckets(df_filter, receiver_position, receiver_regions[receiver_region],compressed=True,only_half=False)
        
        plt.figure()
        plt.subplot(1, 3, 1)
        heatmap = pf.createHeatmap(df_filter.x02, df_filter.y02, df_filter.vast, only_half=True, compressed = True, stat_name = "VAST",
                                   minimum=5,alpha=0.95, vast_min=vast_min,  annot = True, addCount=True)
        pf.plot1D(data=df_filter,alpha_ball = 0, alpha_player= 0.1, only_half=True, compressed=True, receiver_position=receiver_position,
                  title = "Rally VAST: Impact Region (" + impact_region + ") | Receiver " + receiver_position_title + " Region (" + receiver_region +")")
        pf.markupImpactPlayer(impact_region)
        
        plt.subplot(1, 3, 2)
        pf.createKde(df_filter.x02, df_filter.y02,n_levels=15,alpha=1, only_half=True, compressed = True)
        pf.plot1D(data=df_filter, receiver_position=receiver_position, only_half=True, compressed = True, 
                  title="Rally Shots: Impact Region (" + impact_region + ") | Receiver " + receiver_position_title + " Region (" + receiver_region +")")
        pf.markupImpactPlayer(impact_region)
        
        plt.subplot(1, 3, 3)
        if (impact_region == 'Deuce' and receiver_region == "Ad") or (impact_region == 'Ad' and receiver_region == "Deuce"):
            n_clusters = 2
        else:
            n_clusters = 3
        means, covariances, prediction = pf.fitGMM(df_filter,n_clusters,compressed=True,only_half=True,
                                                   title = "Rally Shots: Impact Region (" + impact_region + ") | Receiver " + receiver_position_title + " Region (" + receiver_region +") | GMM")
        pf.markupImpactPlayer(impact_region)
        
        plt.subplots_adjust(top=0.7,bottom=0.2,left=0.04,right=0.97,hspace=0.15,wspace=0.17)
        
        
        # Formatting predictions df
        prediction = pd.DataFrame(prediction)
        prediction = prediction.set_index(df_filter.index)
        prediction = prediction.rename(columns={0:'Dist0', 1: 'Dist1', 2: 'Dist2'})
        # Concatenate dfs
        df_filter_gmm = pd.concat([df_filter,prediction], axis=1)
        # Filter data into rows from each distribution
        df_filter_dist0 = df_filter_gmm[df_filter_gmm.Dist0 > 0.33]
        df_filter_dist1 = df_filter_gmm[df_filter_gmm.Dist1 > 0.33]
        if n_clusters == 2:
            df_filter_gmm = {"Dist0": df_filter_dist0, "Dist1": df_filter_dist1}
        elif n_clusters == 3:
            df_filter_dist2 = df_filter_gmm[df_filter_gmm.Dist2 > 0.33]
            df_filter_gmm = {"Dist0": df_filter_dist0, "Dist1": df_filter_dist1,"Dist2": df_filter_dist2}
    
        plt.figure()
        df_region = df
        df_region = dm.filter_data_by_buckets(df_region, 'impact', impact_regions[impact_region],compressed=True,only_half=False)
        df_region = dm.filter_data_by_buckets(df_region, receiver_position, receiver_regions[receiver_region],compressed=True,only_half=False)
        i = 0
        j = 0
        n_shots = 150
        
        for df_filter_gmm_label in df_filter_gmm:    
            
            df_filter_gmm_specific = df_filter_gmm[df_filter_gmm_label]
            
            plt.subplot(3, 3, 1+i)
            pf.createKde(df_filter_gmm_specific.x02, df_filter_gmm_specific.y02,n_levels=15,alpha=1, only_half=True, compressed = True)
            pf.plot1D(data=df_filter_gmm_specific, receiver_position=receiver_position, only_half=True, compressed = True, 
                      title="Shots from " + df_filter_gmm_label)
            pf.markupImpactPlayer(impact_region)
            plt.scatter(means[i,0],means[i,1],c='orange',zorder=10,s=20)
            #---
            plt.subplot(3, 3, 4+i)
            heatmap = pf.createHeatmap(df_filter_gmm_specific.x02, df_filter_gmm_specific.y02, df_filter_gmm_specific.vast, only_half=True, compressed = True, stat_name = "VAST",
                                       minimum=5,alpha=0.95, vast_min=vast_min,  annot = True, addCount=True)
            pf.plot1D(data=df_filter_gmm_specific,alpha_ball = 0, alpha_player= 0.1, only_half=True, compressed=True, receiver_position=receiver_position,)
            pf.markupImpactPlayer(impact_region)
            plt.scatter(means[i,0],means[i,1],c='orange',zorder=10,s=20)
            bucket_centre = dm.get_peak_vast_heatmap(heatmap)
            plt.scatter(bucket_centre[0],bucket_centre[1],c='blue',zorder=10,s=20)
            #---
            plt.subplot(3, 3, 7+i)
            x,y  = np.random.multivariate_normal(means[j], covariances[j], n_shots).T
            vast_intention = pf.plot_GMM_dist_data(x,y,df_filter,n_shots,'orange') 
            x,y  = np.random.multivariate_normal(bucket_centre, covariances[j], n_shots).T
            vast_rational = pf.plot_GMM_dist_data(x,y,df_filter,n_shots,'blue') 
            
            ax = plt.gca()
            ax.set_xlabel('vast_intention: {:0.2f} | vast_rational: {:0.2f}'.format(vast_intention,vast_rational))
            
            i=i+1
            j=j+1
        
winsound.Beep(2000, 800)

Converged: True
Converged: True
Converged: True
Converged: True
Converged: True
Converged: True
Converged: True
Converged: True
Converged: True


In [6]:
impact_region = 'Deuce'
receiver_region = 'Deuce'

df_filter = df
df_filter = dm.filter_data_by_buckets(df_filter, 'impact', impact_regions[impact_region],compressed=True,only_half=False)
df_filter = dm.filter_data_by_buckets(df_filter, receiver_position, receiver_regions[receiver_region],compressed=True,only_half=False)

X = df_filter[['x02', 'y02']]
X = X.to_numpy()

n_components = np.arange(1, 41)
models = [GaussianMixture(n, covariance_type='full').fit(X)
          for n in n_components]

plt.figure()
plt.plot(n_components, [m.bic(X) for m in models], label='BIC')
plt.plot(n_components, [m.aic(X) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');
plt.title("Cross Court Rally Winning Shots: AIC & BIC Analysis")

Text(0.5, 1.0, 'Cross Court Rally Winning Shots: AIC & BIC Analysis')

In [10]:
n_clusters = 6
plt.figure()
means, covariances, prediction = pf.fitGMM(df_filter,n_clusters,compressed=True,only_half=True,
                                           title = "Rally Shots: Impact Region (" + impact_region + ") | Receiver " + receiver_position_title + " Region (" + receiver_region +") | GMM")
pf.markupImpactPlayer(impact_region)

Converged: True


In [11]:
# Formatting predictions df
prediction = pd.DataFrame(prediction)
prediction = prediction.set_index(df_filter.index)
prediction = prediction.rename(columns={0:'Dist0', 1: 'Dist1', 2: 'Dist2',3:'Dist3', 4: 'Dist4', 5: 'Dist5'})
# Concatenate dfs
df_filter_gmm = pd.concat([df_filter,prediction], axis=1)
# Filter data into rows from each distribution
df_filter_dist0 = df_filter_gmm[df_filter_gmm.Dist0 > 0.33]
df_filter_dist1 = df_filter_gmm[df_filter_gmm.Dist1 > 0.33]
df_filter_dist2 = df_filter_gmm[df_filter_gmm.Dist2 > 0.33]
df_filter_dist3 = df_filter_gmm[df_filter_gmm.Dist3 > 0.33]
df_filter_dist4 = df_filter_gmm[df_filter_gmm.Dist4 > 0.33]
df_filter_dist5 = df_filter_gmm[df_filter_gmm.Dist5 > 0.33]


df_filter_gmm = {"Dist0": df_filter_dist0, "Dist1": df_filter_dist1,"Dist2": df_filter_dist2,
                 "Dist3": df_filter_dist3, "Dist4": df_filter_dist4,"Dist5": df_filter_dist5,}

df_region = df
df_region = dm.filter_data_by_buckets(df_region, 'impact', impact_regions[impact_region])
df_region = dm.filter_data_by_buckets(df_region, receiver_position, receiver_regions[receiver_region])

n_shots = 150


In [12]:
plt.figure()
i = 1
j = 0
for df_filter_gmm_label in df_filter_gmm:    
    
    df_filter_gmm_specific = df_filter_gmm[df_filter_gmm_label]
    
    # plt.subplot(3, 7, 1+i)
    # pf.createKde(df_filter_gmm_specific.x02, df_filter_gmm_specific.y02,n_levels=15,alpha=1, only_half=True, compressed = True)
    # pf.plot1D(data=df_filter_gmm_specific, receiver_position=receiver_position, only_half=True, compressed = True, 
    #           title="Shots from " + df_filter_gmm_label)
    # pf.markupImpactPlayer(impact_region)
    # plt.scatter(means[i,0],means[i,1],c='orange',zorder=10,s=20)
    # #---
    # plt.subplot(3, 7, 8+i)
    plt.subplot(3, 4, i)
    heatmap = pf.createHeatmap(df_filter_gmm_specific.x02, df_filter_gmm_specific.y02, df_filter_gmm_specific.vast, only_half=True, compressed = True, stat_name = "VAST",
                               minimum=5,alpha=0.95, vast_min=vast_min,  annot = True, addCount=True, title=df_filter_gmm_label)
    # pf.plot1D(data=df_filter_gmm_specific,alpha_ball = 0, alpha_player= 0.1, only_half=True, compressed=True, receiver_position=receiver_position,)
    # pf.markupImpactPlayer(impact_region)
    plt.scatter(means[j,0],means[j,1],c='orange',zorder=10,s=20)
    bucket_centre = dm.get_peak_vast_heatmap(heatmap)
    plt.scatter(bucket_centre[0],bucket_centre[1],c='blue',zorder=10,s=20)
    #---
    # plt.subplot(3, 7, 15+i)
    plt.subplot(3, 4, i+1)
    x,y  = np.random.multivariate_normal(means[j], covariances[j], n_shots).T
    vast_intention = pf.plot_GMM_dist_data(x,y,df_filter,n_shots,'orange', title=df_filter_gmm_label) 
    x,y  = np.random.multivariate_normal(bucket_centre, covariances[j], n_shots).T
    vast_rational = pf.plot_GMM_dist_data(x,y,df_filter,n_shots,'blue',title=df_filter_gmm_label) 
    
    ax = plt.gca()
    ax.set_xlabel('vast_intention: {:0.2f} | vast_rational: {:0.2f}'.format(vast_intention,vast_rational))
    
    i=i+2
    j=j+1

plt.subplots_adjust(top=0.945,bottom=0.07,left=0.04,right=0.98,hspace=0.41,wspace=0.2)
